Ta dokument je namenjen testiranju in ucenju uporabe NER-a, saj se prvic srecujem z njim.

**uporaba CLASSLA**

V drugo sem imel vec srece, classla namrec podpira slovenski jezik tudi za NER.

In [1]:
import classla

In [2]:
classla.download("sl")

2025-06-01 22:10:21 INFO: Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2025-06-01 22:11:07 INFO: File exists: /data/home/jasa/classla_resources/sl/pos/standard.pt.
2025-06-01 22:11:08 INFO: File exists: /data/home/jasa/classla_resources/sl/lemma/standard.pt.
2025-06-01 22:11:09 INFO: File exists: /data/home/jasa/classla_resources/sl/depparse/standard.pt.
2025-06-01 22:11:10 INFO: File exists: /data/home/jasa/classla_resources/sl/ner/standard.pt.
2025-06-01 22:11:12 INFO: File exists: /data/home/jasa/classla_resources/sl/pretrain/standard.pt.
2025-06-01 22:11:12 INFO: Finished downloading models and saved to /data/home/jasa/classla_resources.


In [3]:
nlp = classla.Pipeline("sl", processors="tokenize,pos,lemma,ner", use_gpu=True)

doc = nlp("Predsednik vlade Robert Golob je obiskal Novo Gorico.")

for ent in doc.ents:
    print(ent.text, ent.type)

2025-06-01 22:11:12 INFO: Loading these models for language: sl (Slovenian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| ner       | standard |

2025-06-01 22:11:13 INFO: Use device: gpu
2025-06-01 22:11:13 INFO: Loading: tokenize
2025-06-01 22:11:13 INFO: Loading: pos
2025-06-01 22:11:56 INFO: Loading: lemma
2025-06-01 22:12:15 INFO: Loading: ner
2025-06-01 22:12:15 INFO: Done loading processors!


Robert Golob PER
Novo Gorico LOC


**Zgoraj se je classla dobro obnesla, a se je hitro zataknilo**

In [4]:
import os
import json

nastavimo podatke za direktori

In [5]:
input_folder = './json-test/'

nalozimo clanke v pomnilnik:

In [6]:
clanki = []
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        file_path = os.path.join(input_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                if isinstance(data, list):
                    clanki.extend(data)
                else:
                    clanki.append(data)
            except json.JSONDecodeError as e:
                print(f"Napaka pri branju {filename}: {e}")

hitra testna koda:

In [7]:
for i, clanek in enumerate(clanki, 1):
    naslov = clanek.get("title", f"Članek {i}")
    besedilo = clanek.get("text", "")

    doc = nlp(besedilo)

    imena = set()
    lokacije = set()

    for ent in doc.ents:
        if ent.type == "PER":
            imena.add(ent.text)
        elif ent.type == "LOC":
            lokacije.add(ent.text)

    print(f"\n=== {naslov} ===")
    if imena:
        print("Imena:")
        for ime in sorted(imena):
            print(f" - {ime}")
    else:
        print("Imena: (ni najdenih)")

    if lokacije:
        print("Lokacije:")
        for lok in sorted(lokacije):
            print(f" - {lok}")
    else:
        print("Lokacije: (ni najdenih)")


=== Predsednik vlade Robert Golob je obiskal Novo Gorico. ===
Imena:
 - Robert Golob
Lokacije:
 - Novo Gorico

=== Prometno nesrečo, v kateri je umrlo 12 ljudi, povzročil tovornjak ===
Imena:
 -  je včeraj pretr
 - eri je umrlo 12
 - orjen tovornjak
Lokacije:
 - Bratislava
 - Nesre
 - Nitra
 - Slovaška
 - Slovaško
 - e umr
 - ometna ne

=== Še vedno ni jasno, kdo bo plačal 250 tisoč evrov za zdravljenje beguncev in migrantov ===
Imena:
 -  bl
 -  da se zdr
 -  vedno pou
 - , a
 - agajne, še
 - ajo na Zav
 - ali so ga 
 - eno
 - ern
 - ga vsako l
 - k predstav
 - n n
 - o zaradi b
 - r namenjen
 - rše
 - stveno zav
 - trs
 - una
 - van
 - vi 
 - zdr
Lokacije:
 - , ki ga v
 - Slovenijo
 - be, ki pr

=== 'Če zmaga, bo to dokaz, da imajo Slovaki dovolj korupcije' ===
Imena:
 -  v zadnji
 -  zadnjih
 - ,4 milijo
 - Andrej Kiska
 - Jana Kuciaka
 - Marošem Šefčovičem
 - Zuzana Čaputova
 - anji p
 - bri lj
 - e storij
 - glasna a
 - glasna aktiv
 - i ljudje
 - i v drug
 - ivcev,
 - ka proti k

**Rezultat**
razocaranje, imena zaznava katastrofalno, ker bi bilo mozno da se pojavljajo cudni znaki, bom probal seenkrat in sicer z ociscenim besedilom.

Koda za ciscenje besedila:

In [8]:
import re

def ocisti_besedilo(besedilo):
    besedilo = re.sub(r"\s+", " ", besedilo)  # Odvečne beline
    besedilo = re.sub(r"[^\w\s.,!?žščćčŠŽČĆ]", "", besedilo)  # Neobičajni znaki
    return besedilo.strip()

Ponovno pozenimo enako kodo kot zgoraj:

In [9]:
for i, clanek in enumerate(clanki, 1):
    naslov = clanek.get("title", f"Članek {i}")
    besedilo = clanek.get("text", "")

    doc = nlp(ocisti_besedilo(besedilo))

    imena = set()
    lokacije = set()

    for ent in doc.ents:
        if ent.type == "PER":
            imena.add(ent.text)
        elif ent.type == "LOC":
            lokacije.add(ent.text)

    print(f"\n=== {naslov} ===")
    if imena:
        print("Imena:")
        for ime in sorted(imena):
            print(f" - {ime}")
    else:
        print("Imena: (ni najdenih)")

    if lokacije:
        print("Lokacije:")
        for lok in sorted(lokacije):
            print(f" - {lok}")
    else:
        print("Lokacije: (ni najdenih)")


=== Predsednik vlade Robert Golob je obiskal Novo Gorico. ===
Imena:
 - Robert Golob
Lokacije:
 - Novo Gorico

=== Prometno nesrečo, v kateri je umrlo 12 ljudi, povzročil tovornjak ===
Imena:
 - Peter Pellegrini
 - Zuzana Čaputova
Lokacije:
 - Bratislava
 - Cipru
 - Košic
 - Nitra
 - Slovaška
 - Slovaškem
 - Slovaško

=== Še vedno ni jasno, kdo bo plačal 250 tisoč evrov za zdravljenje beguncev in migrantov ===
Imena:
 - Damjan Kos
 - Kos
 - Samo Fakin
Lokacije:
 - Sloveniji
 - Slovenijo

=== 'Če zmaga, bo to dokaz, da imajo Slovaki dovolj korupcije' ===
Imena:
 - Andrej Kiska
 - Dennik N.
 - Jana Kuciaka
 - Macron
 - Macron Če Čaputova
 - Marošem Šefčovičem
 - Matus Kostolny
 - Milan Nic
 - Roberta Fica
 - Slovakov
 - Zuzana Čaputova
 - Čaputova
 - Čaputovo
Lokacije:
 - Evrope
 - Madžarskem
 - Poljskem
 - Slovaško

=== Bodo Slovaki prvič v zgodovini izvolili žensko predsednico? ===
Imena:
 - Andrej Kiska
 - Jana Kuciaka
 - Maroš Šefčovič
 - Roberta Fica
 - Zuzana Čaputova
 - Zuzano Ča

**Rezultat** <br>
Pozitivno presenecenje, dosti bolje, morda celo dovolj. <br> Naletel sem na nov problem, in sicer imena so podvojena recimo Zuzana Caputova in Zuzano Caputovo, zato sem kodi dodal ze lemma, kar naj bi jo standariziral

In [10]:
for i, clanek in enumerate(clanki, 1):
    naslov = clanek.get("title", f"Članek {i}")
    besedilo = clanek.get("text", "")

    doc = nlp(ocisti_besedilo(besedilo))

    imena = set()
    lokacije = set()

    for ent in doc.ents:
        if ent.type == "PER":
            # Zberi leme vseh tokenov v entiteti
            leme = []
            for sent in doc.sentences:
                for token in sent.tokens:
                    if token.start_char >= ent.start_char and token.end_char <= ent.end_char:
                        leme.append(token.words[0].lemma)
            imena.add(" ".join(leme))

        elif ent.type == "LOC":
            lokacije.add(ent.text)

    print(f"\n=== {naslov} ===")
    if imena:
        print("Imena:")
        for ime in sorted(imena):
            print(f" - {ime}")
    else:
        print("Imena: (ni najdenih)")

    if lokacije:
        print("Lokacije:")
        for lok in sorted(lokacije):
            print(f" - {lok}")
    else:
        print("Lokacije: (ni najdenih)")


=== Predsednik vlade Robert Golob je obiskal Novo Gorico. ===
Imena:
 - Robert Golob
Lokacije:
 - Novo Gorico

=== Prometno nesrečo, v kateri je umrlo 12 ljudi, povzročil tovornjak ===
Imena:
 - Peter Pellegrini
 - Zuzana Čaputov
Lokacije:
 - Bratislava
 - Cipru
 - Košic
 - Nitra
 - Slovaška
 - Slovaškem
 - Slovaško

=== Še vedno ni jasno, kdo bo plačal 250 tisoč evrov za zdravljenje beguncev in migrantov ===
Imena:
 - Damjan Kos
 - Kos
 - Samo Fakin
Lokacije:
 - Sloveniji
 - Slovenijo

=== 'Če zmaga, bo to dokaz, da imajo Slovaki dovolj korupcije' ===
Imena:
 - Andrej Kiska
 - Dennik n.
 - Jan Kuciak
 - Macron
 - Macron če Čaputov
 - Maroš Šefčovič
 - Matus Kostolny
 - Milan Nic
 - Robert Fico
 - Slovak
 - Zuzana Čaputov
 - Čaputov
Lokacije:
 - Evrope
 - Madžarskem
 - Poljskem
 - Slovaško

=== Bodo Slovaki prvič v zgodovini izvolili žensko predsednico? ===
Imena:
 - Andrej Kiska
 - Jan Kuciak
 - Maroš Šefčovič
 - Robert Fico
 - Zuzana Čaputov
 - Čaputov
Lokacije:
 - Slovaška
 - Slova

**Tole mi je ze kar vsec** <br> nadaljujem tako da bo dejansko zmodificiran osnovni .json <br> ponovno nalozim datoteko

In [11]:
from collections import Counter
import time

In [12]:
with open('combined_test.json', 'r', encoding='utf-8') as f:
    clanki = json.load(f)

Spodnji for loop je nadgrajena verzija zgornjega, nad nalozenimi clanki namrec pozene prepoznavo entitet tako za osebe, lokacije ter organizacije, dobljene rezultate doda jsonu o clanku, pri tem si shrani tudi stevilo ponovitev.

In [13]:
start = time.time()
for i, clanek in enumerate(clanki, 1):
    naslov = clanek.get("title", f"Članek {i}")
    besedilo = clanek.get("text", "")

    doc = nlp(ocisti_besedilo(besedilo))

    osebe = Counter()
    lokacije = Counter()
    organizacije = Counter()

    for ent in doc.ents:
        leme = []
        for sent in doc.sentences:
            for token in sent.tokens:
                if token.start_char >= ent.start_char and token.end_char <= ent.end_char:
                    leme.append(token.words[0].lemma)
        besedilo_entitete = " ".join(leme) if leme else ent.text

        if ent.type == "PER":
            osebe[besedilo_entitete] += 1
        elif ent.type == "LOC":
            lokacije[besedilo_entitete] += 1
        elif ent.type == "ORG":
            organizacije[besedilo_entitete] += 1

    # Shrani vse v članek
    clanek["persons"] = dict(osebe)
    clanek["locations"] = dict(lokacije)
    clanek["organizations"] = dict(organizacije)

stop = time.time()
print(f"Za obdelavo {len(clanki)} clankov smo porabili: {stop-start:.0f} sekund")
print(f"Predviden cas za kategorizacijo vseh clankov: {(stop-start)*306000/(len(clanki)):.0f} sekund oz. {(stop-start)*306000/(len(clanki))/60:.0f} minut")

Za obdelavo 55 clankov smo porabili: 46 sekund
Predviden cas za kategorizacijo vseh clankov: 256981 sekund oz. 4283 minut


**Predolgo traja** <br>
skratka, ker classla tece le ne procesorju imamo en micken problemcic, rece se mu cas, prej bom umru kukr se bo to koncalo <br><br> 
**Resitev**
multithreding, glej dokument ClasslaMultithr.py

testiram kako deluje:

In [14]:
for i, clanek in enumerate(clanki, 1):
    naslov = clanek.get("title", "")
    persons = clanek.get("persons")
    locations = clanek.get("locations")
    organizations = clanek.get("organizations")
    print(f"{i}.: {naslov}")
    print(f"   Persons: {persons}")
    print(f"   Locations.: {locations}")
    print(f"   Organizations.: {organizations}")

1.: Biden za vodjo prometnega resorja izbral Peta Buttigiega
   Persons: {'Buttigieg': 4, 'Biden': 1, 'Rahm Emanuel': 2, 'Eric Garcetti': 2, 'Rhode Island Gina Raimondo': 2, 'Buttgieg': 1, 'Bidn': 1, 'Beau Bidn': 1, 'Joe Biden': 1, 'South Benda': 2}
   Locations.: {'Chicago': 2, 'Los Angeles': 2, 'Dallas': 1}
   Organizations.: {'združen narod': 2, 'South bend tribuna': 2}
2.: Kaj je koronavirus in kakšni so simptomi okužbe?
   Persons: {}
   Locations.: {'Kitajska': 3, 'protivirusen': 5}
   Organizations.: {}
3.: Kim Džong Un znova provocira. Zaradi jedrske eksplozije Obama zagrozil z novimi sankcijami
   Persons: {'Yonhap': 2, 'Šinzo Abe': 2, 'Barack Obama': 2, 'Obama': 9, 'park Geun Hye': 3, 'Shinzo Abe': 3, 'Cho Taeyong': 2, 'Francois Hollande': 1, 'Yukiya Amano': 2}
   Locations.: {'južen Koreja': 6, 'severen Koreja': 34, 'Japonska': 7, 'Kitajska': 2, 'francijajaponski': 1, 'Pjongjang': 9, 'Peking': 5, 'Korej': 1, 'Rusija': 1, 'ZDA': 1, 'Kitajsko': 3, 'japonski morje': 1, 'Sloveni